In [1]:
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive, HBox, VBox
from ascii_art_conversion_simple import img2ascii_arr
from ascii_art_conversion_v2 import img2ascii_arr_v2
from img_processing import preprocess_img, DITHER_MODES
from braille_art import img2braille_arr
from ascii_palettes import ascii_palette_rand, ascii_palette_v2_rand

In [2]:
orig_img = Image.open("imgs/irad_grad.bmp").convert("L")
#orig_img = ImageOps.invert(orig_img)

print(orig_img.size)

(512, 512)


In [3]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq, dither):
    proc_img = preprocess_img(orig_img, scale_factor, contrast, brightness, eq, quantize_colors, dither)
    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(9, 3))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            eq=(0, 1, 0.01),
            quantize_colors=(1, 256, 1),
            dither=DITHER_MODES)

controls = VBox(interactive_preprocess.children[:-1])
display(HBox((controls, interactive_preprocess.children[-1])))

In [12]:
proc_img = interactive_preprocess.result

In [13]:
ascii_arr = img2ascii_arr(proc_img, scale_vertically=True, palette=ascii_palette_rand, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

   ....:``:,-;;;-_;"-^;~^_!^-^;!-;--;_:_::'`....   
 ..:'::'-;-;;L""L^cL>l>1cl1===^1Ll"!!";-;;-,```,...
.:'''__;-_"!"Lc^=>+=l+(>+v>l1Y>1=c>cLcL~~^_;_;,:':.
.,',--_!!~c1=c=fv(Tv/fr(E|<Cnv<Fs/n=|+=1!!"^_-;-`:,
`,;_;"^~^1l=1|n7xxuSkbCkJI[PXbatosisi+s+1=LL~"__;,'
:--!~"!l=+1xoxrn)K}kRkR5ywHHpkDII[PIvF?/ljc>>"^!;__
;;_"!Ll+=F7nY{*bt5ywwDODR8HAH8w5ADE4kXvu7s1c=+"!~;;
--!Lcl>vs(uX3bA6HyGy888d09Og#%HNH$wDD[bVi?|Y>1=LL__
_""=+c=fFTeC]HpD%y89B&9QGW#WM9W0%y%5yH*]4[fj+llL^!^
_^Lc=lvTnaStR550O8&QWQWQWWWWWWG#0%&0DwRpZ*rj</clc~"
~!l=c</FV3CApyG%O#WGQWW@W@W@QWQWWd%g%p6whhk(f=llc^L
;L+c=|uzPhywA#pq&QBWW@W@Q@@Q@QQQQQmM0Dy5H*{]f7?+>~!
^~c1=7sfX}2wAMOqOQQQWW@W@@@W@W@W9Q$dOdD6HXho7T=++!^
_"l+ljio24yDR5B%dmQQW@Q@@W@@WWQWWB&BBp6w6Ia)(i==1"!
!"++=?ziIZ}HHp&8B%WWQWWQW@0Q@QWQMQ%N6OyDEeXuY7lcl!L
;!"=+lisj}bHpw6mB8O8MQQWWQQW%Q#%$B$6&Rpw3hPfjc1>1!L
-~"lcc\ur[t4Pw6yp##&9&9OGWNW9OO#MMypDw{Z[Evvx>1+^!_
;"~Lll>>nYzZ4KAywAy0D%qGNG8MBNg5GAHw6bVbUj7+>c+^^";
'-^~!1lc\lr\J3{)yDpARw%ABAAwypRwRA6hXtKosrcF+1~~~_;
,__~L^"+>c|s

In [21]:
ascii_arr = img2ascii_arr_v2(proc_img, palette=ascii_palette_v2_rand, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

    .:::::;:;;;;;;;;;;;L;L;L;;;;;;;;;;;:;::::. .   
 .::::::;;;;;;~;lcc===ccc==1l1==c1L!;;;;;;;:::::.  
.:.::;;;;;;!cL==Ll=x1svvzxzslvls1l1lclL~;;;;;:::::.
:::;:;;;~~cLlclz1ooooijkkJkkzkzvozscslll1L;;;;;::::
::;;;;~;1Lc=xsxzkjJJJjJJJJJJJJjkkijvzn11cc1L~;;;;::
:;;;;!1==1slnjikJJJJJwwwwwwwwwJJJJJjijvocl1Ll;!;;;:
;;;~~=L=s=xjjjJJJwwwwwwwwwwwwwwwwwJJJkJksn1==cL;;;;
;;!LcLlocJkJJJJwwww&O&&&&&&&&&&d&wwwwJJjkjxolll=;;;
;;!=llvckjJJJwwww&&&&&&W&W&&&&&&d&OwwwwJJiJvclLl;!;
;~=L1csJJJJJwww&&G&&WQQQQQQQQW&&&&&GwwwwJJjin=l=l;;
;1LlcsjJiJJwwdw&&&WQQQQQQQQQQQQQW&&&dwwwwkJJvv11L~;
;~1ccvjjJJwwww&&&WQQQQQ@@@@@QQQQQ&W&&dwwwkJJknlllL!
;cLlsljJJJwwOw&&W&QQQ@@@@@@@@QQQQW&&&wdwwwJjks=clL;
;l=l1oJiiJwwwd&&&WQQQQQ@@@@Q@QQQQ&W&d&ww)wjJjz1==c;
;!L==vjjkJwwwwO&&&WQQQQQQQQQQQQQW&&&GwGwwjJkks=1L~;
;!ccllxijJUwwwO&d&&WWQQQQQQQQQW&W&&G&wwhwikjxl1l1;~
;;lLl=skjkkZwwwdmG&&&WWWQWQWW&W&&G&OwwwAJkkkxclL=;;
;;~~11csjikJ2Zwwwdd&d&&&&&&&&&&G&wwwwb)JkJJoc=lL;!;
-;;~Lcl=xkkkkj2wAwwwGwOGO&GOOGwwGwwb)Jikkoc=ccL!;;;
-;;!;lL=l=xJ

In [15]:
ascii_arr = img2braille_arr(proc_img, dither=DITHER_MODES.FS)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

⠀⠀⠠⢀⠂⠔⡐⡐⢔⢐⢅⠢⡊⢔⡐⢅⠢⡂⡂⡂⡂⠄⠂⡀⠀
⠀⠌⡐⠄⢅⠕⡐⡅⡕⡌⡆⡇⡣⡣⡪⢪⢨⢢⠱⡐⢌⠌⡐⠠⢁
⠈⠔⡨⠨⡢⡑⡅⡇⣕⢕⡕⡵⡱⣕⢵⡱⡕⡕⡕⠥⡑⡌⢌⠌⢄
⢈⠢⡊⢜⠔⡕⡕⡵⡕⡧⣫⣞⣝⢮⣺⢜⡮⡪⡎⡇⡇⡕⢅⠪⡐
⢐⢅⠪⡊⡎⡮⡺⡪⡯⣺⣳⣳⣽⢽⣺⣳⢽⣹⡪⣺⢸⢸⢨⠢⡊
⠰⡐⢕⢕⢕⢵⢝⡽⡽⡽⣾⣽⣾⣿⣻⢾⡽⣺⢺⡪⡎⡎⡆⡣⢊
⠨⠪⡸⢸⢸⢪⡳⡽⣝⣯⢿⣽⣿⣯⣿⢯⡯⣗⢯⡺⡪⡪⡪⡊⡢
⠨⡱⡘⡜⡜⣕⢝⢮⢗⡯⣿⢽⣾⣻⢾⢯⢯⢯⡳⡹⡜⡕⢕⢌⠢
⠨⢂⢪⠸⡸⣘⢎⡏⣗⢯⢯⣻⡺⡽⡽⡽⣝⢵⡹⡪⡪⡪⡊⡢⢑
⢈⠢⠡⡱⡘⡔⡕⡕⣇⢯⢺⢜⡞⣝⣕⢯⢺⢸⡸⡸⡸⡐⢕⢈⢂
⢀⠊⠌⠔⢌⠢⡃⡇⡣⡣⡳⡱⡹⡸⡰⡕⡕⡕⡪⡊⡢⢊⠔⡐⠐
⠀⠄⠡⢁⠢⢑⠌⡢⠱⡘⡌⡪⡊⢎⠪⡘⢌⠢⡑⠔⡨⢐⠐⡀⠅
